In [5]:
library("btergm")
library("dplyr")
library("Matrix")
library("statnet")  
library("texreg") 
library("data.table")
library("plyr")
library("metafor")
library("dotwhisker")


Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)


In [6]:
#Data Preparation 

#Generate edge matrix and adjust for missing players
create_round_edges <- function(data, players_in_game){
  players <- length(players_in_game)
  edge_mat <- matrix(0,nrow=players,ncol=players)
  data <- adjust_missing_players(data, players_in_game)
  for(j in 1:nrow(data)){
    player_id <- data[[j, "player_id"]]
    other_id <- data[[j, "other_id"]]
    if(player_id != other_id){
      edge_mat[player_id, other_id] <- 1
    }
  }
  return(edge_mat)
}

#Add Endowment as a vertex attribute to the network
add_endowment <- function(network, data){
  endowment <- unique(data[(data$round==1), ])
  endowment <- endowment[ , c("player_id", "endowment")][order(endowment$player_id), ]
  network <- set.vertex.attribute(network, "endowment", endowment[,2])
  return(network)
}

#Adjust wealth for missing players and add it to the network
add_wealth <- function(network, data, round){
  wealth_data <- as.data.table(data)
  wealth_data <- wealth_data[ , wealth := shift(final_wealth, fill=0), by=c('treatment', 'group_id', 'player_id')]
  #wealth_data <- transform(wealth_data, wealth=sqrt(wealth))
  wealth_data <- ddply(wealth_data, 'round', transform, wealth = scale(wealth, scale=TRUE)) 
  wealth_data$wealth[wealth_data$round==1] <- 0
  wealth_data$wealth <- as.numeric(wealth_data$wealth)
  players_in_game <- wealth_data[(wealth_data$round==1), ]$player_id
  wealth_data <- adjust_missing_players(wealth_data, players_in_game)
  setDF(wealth_data)
  round_wealth <- get_round_wealth(wealth_data, round)
  network <- set.vertex.attribute(network, "wealth", round_wealth)
  return(network)
}

#Adjust ID's to account for missing players
adjust_missing_players <- function(data, players_in_game){
  num_players <- length(players_in_game)
  if (num_players < max(players_in_game)) {
    x <- setdiff(seq(1, max(players_in_game)), players_in_game) 
    data$player_id[data$player_id > x] <- data$player_id[data$player_id > x] - 1
    if("other_id" %in% colnames(data))
    {
      data$other_id[data$other_id > x] <- data$other_id[data$other_id > x] - 1
    }
  }
  return(data)
}

#Returns the wealth of a player for in a specific round
get_round_wealth <- function(data, current_round){
  round_data <- data[(data$round==current_round), ]
  return(round_data[,"wealth"])
}

#Generate the network in the correct format and add the outdegree as a vertex atrribute
get_btergm_data <- function(data1, data2){
  network <- list()
  players_in_game <- union(unique(data1$player_id), unique(data1$other_id))
  num_players <- length(players_in_game)
  for(i in 1:20){
    round_edges <- create_round_edges(data1[(data1$round == i), ], players_in_game)
    network[[i]] <- network(round_edges)
    network[[i]] <- add_endowment(network[[i]], data2)
    network[[i]] <- add_wealth(network[[i]], data2, i)
    outdegree <- degree(network[[i]], cmode = "outdegree")
    network[[i]] <- set.vertex.attribute(network[[i]], "outdegree", outdegree)
  }
  return(network)
}

#   Metafor package - not used anymore

#Extract model data for metafor
get_metafor_data <- function(group_models){
  terms <- btergm.se(group_models[[1]])
  yis <-matrix(list(), nrow=length(group_models), ncol=nrow(terms))
  seis <- matrix(list(), nrow=length(group_models), ncol=nrow(terms))
  for(i in 1:length(group_models)){
    model <- group_models[[i]]
    stats <- btergm.se(model)
    for(j in 1:nrow(stats)){
      yis[i,j] = stats[j,1] #Gets the coef estimate
      seis[i,j] = stats[j,2] #Gets the coef standart error
    }
  }
  return(export_coefs(btergm08(yis,seis,terms),terms))
}


#Run meta-analysis using all group models for each coefficient
btergm08 <- function(yis,seis,terms){
  yis <- split(yis, rep(1:ncol(yis), each = nrow(yis)))
  seis <- split(seis, rep(1:ncol(seis), each = nrow(seis)))
  coefs <- list()
  for(i in 1:nrow(terms)){
    yi <- yis[i]
    sei <- seis[i]
    yi<- unlist(yi, use.names=FALSE)
    sei<- unlist(sei, use.names=FALSE)
    coef <- rownames(terms)[i]
    new.term <- data.frame(yi, sei)
    meta<- rma.uni(yi, sei, data = new.term, method = "FE", level = 95)
    coefs[[coef]] <- meta
  }
  return (coefs)
}

#Create coefficient matrix
export_coefs <- function(coefs,terms){
  mat <- matrix(list(), nrow = nrow(terms), ncol = 5)
  rownames(mat) <- rownames(terms)
  for(name in rownames(terms)){
    term <- coefs[[name]]
    est <- as.numeric(term["beta"])
    se <- as.numeric(term["se"])
    lb <-as.numeric(term["ci.lb"])
    ub <-as.numeric(term["ci.ub"])
    lev <- 1-as.numeric(term["level"])
    mat[name,] <- c(est,se,lb,ub,lev)
  }
  colnames(mat) <- c("estimate","se","mu-min","mu-plus","alpha-mu")
  return(mat)
}

#Test function
summarize_models <- function(models1, models2, models3){
  summary_model1 <- get_pooled_model(models1)
  summary_model2 <- get_pooled_model(models2)
  summary_model3 <- get_pooled_model(models3)
  write.csv(summary_model1, paste('btergm_version2_params_', '.csv', sep=''))
  write.csv(summary_model2, paste('btergm_no_edges_params_', '.csv', sep=''))
  write.csv(summary_model3, paste('btergm_no_edges_nodematch_params_', '.csv', sep=''))
  #Test
  test <- merge(summary_model1,summary_model2,by="row.names",all.x=TRUE)
  test <- test[, 2:11]
  rownames(test) <- rownames(summary_model1)
  test <- merge(test,summary_model3,by="row.names",all.x=TRUE)
  return(test)
}

#End - Metafor package

get_pooled_model <- function(group_models){
  terms <- btergm.se(group_models[[1]])
  yis <-matrix(list(), nrow=length(group_models), ncol=nrow(terms))
  seis <- matrix(list(), nrow=length(group_models), ncol=nrow(terms))
  numobs <- c()
  for(i in 1:length(group_models)){
    model <- group_models[[i]]
    stats <- btergm.se(model)
    numobs[i] <- as.numeric(nobs(model)[2])
    for(j in 1:nrow(stats)){
      yis[i,j] = stats[j,1] #Gets the coef estimate
      seis[i,j] = stats[j,2] #Gets the coef standart error
    }
  }
  combine_models(yis,seis,numobs,terms)
}

combine_models <- function(yis,seis,numobs,terms){
  yis <- split(yis, rep(1:ncol(yis), each = nrow(yis)))
  seis <- split(seis, rep(1:ncol(seis), each = nrow(seis)))
  pooled_effs <- list()
  pooled_se <- list()
  for(i in 1:nrow(terms)){
    yi <- yis[i]
    sei <- seis[i]
    n <- numobs[i]
    yi<- unlist(yi, use.names=FALSE)
    sei<- unlist(sei, use.names=FALSE)
    pooled_effs[i] <- mean(yi)
    pooled_se[i] <- get_pooled_SE(sei,numobs)
  }
  return(export_coefs2(pooled_effs,pooled_se, terms))
}

export_coefs2 <- function(pooled_effs,pooled_se,terms){
  mat <- matrix(list(), nrow = nrow(terms), ncol = 5)
  rownames(mat) <- rownames(terms)
  for(i in 1:nrow(terms)){
    est <- as.numeric(pooled_effs[i])
    se <- as.numeric(pooled_se[i])
    lb <-est-1.96*se
    ub <-est+1.96*se
    lev <- 0.95
    mat[i,] <- c(est,se,lb,ub,lev)
  }
  colnames(mat) <- c("estimate","std.error","mu-min","mu-plus","alpha-mu")
  return(mat)
}


#Helper SE func 
get_pooled_SE <- function(sei,numobs){
  numer <- 0
  denom <-0
  for(i in 1:length(sei)){
    n <- numobs[i]
    sd <- sei[i]*sqrt(numobs[i])
    sdpwd <-(sd^2)
    numer <- numer + (n-1)*sdpwd
    denom <- denom + n - 1
  }
  return(sqrt(numer/denom)/sqrt((denom + length(sei))))
}



export_model <- function(model, model_name){
  summary <- get_pooled_model(model)
  df <- data.frame(matrix(unlist(summary), ncol=length(summary)))
  write.csv(summary, paste('btergm_output/', model_name, '.csv', sep=''))
  
}

#Used to create a delrecip/wealth matrix
get_wealth <- function(data, round){
  wealth_data <- as.data.table(data)
  wealth_data <- wealth_data[ , wealth := shift(final_wealth, fill=0), by=c('treatment', 'group_id', 'player_id')]
  wealth_data <- ddply(wealth_data, 'round', transform, wealth = scale(wealth, scale=TRUE)) 
  wealth_data$wealth[wealth_data$round==1] <- 0
  wealth_data$wealth <- as.numeric(wealth_data$wealth)
  players_in_game <- wealth_data[(wealth_data$round==1), ]$player_id
  wealth_data <- adjust_missing_players(wealth_data, players_in_game)
  setDF(wealth_data)
  round_wealth <- get_round_wealth(wealth_data, round)
  return(round_wealth)
}


#Create wealth x delrecip covariate 
create_wealth_delrecip_cov <- function(data1, data2){
  matrices <- list()
  players_in_game <- union(unique(data1$player_id), unique(data1$other_id))
  num_players <- length(players_in_game)
  for(i in 1:20){
    round_edges <- create_round_edges(data1[(data1$round == i), ], players_in_game)  
    scaled_wealth <- get_wealth(data2, min(i+1,20))
    round_edges <- transform_ties(round_edges,scaled_wealth)
    matrices[[i]] <- round_edges
  }
  return(matrices)
}

#Tranfrom Ties depending on i's and j's wealth and return the transposed matrix
transform_ties <- function(matrix, scaled_wealth){
  for(i in 1:nrow(matrix)){
    for(j in 1:ncol(matrix)){
      if(matrix[i,j] == 1){
        if(scaled_wealth[i] >= scaled_wealth[j]){
          matrix[i,j] = 1
        }else{
          matrix[i,j] =-1 
        }
      }
    }
  }
  return(t(matrix))
}

get_endowment <- function(data){
  endowment <- unique(data[(data$round==1), ])
  endowment <- endowment[ ,c("player_id", "endowment")][order(endowment$player_id), ]
  return(endowment)
}
#Tranfrom Ties depending on i's and j's endow and return the transposed matrix
transform_ties_endow <- function(matrix, endow){
  for(i in 1:nrow(matrix)){
    for(j in 1:ncol(matrix)){
      if(matrix[i,j] == 1){
        if(endow[i,"endowment"] > endow[j,"endowment"]){
          matrix[i,j] = 1
        }else if(endow[i,"endowment"] < endow[j,"endowment"]){
          matrix[i,j] = -1 
        }else{
          matrix[i,j] = 0  
        }
      }
    }
  }
  return(t(matrix))
}

edgecov_add_lag <- function(edgecov, lag){
  rows <- nrow(edgecov[[1]])
  cols <- ncol(edgecov[[1]]) 
  mat <- matrix(0,nrow=rows,ncol=cols)
  for(i in 1:lag){
    edgecov <- append(list(mat),edgecov)
  }
  return(edgecov[1:20])
}

#Create endow x delrecip covariate 
create_endow_delrecip_cov <- function(data1, data2){
  matrices <- list()
  players_in_game <- union(unique(data1$player_id), unique(data1$other_id))
  num_players <- length(players_in_game)
  for(i in 1:20){
    round_edges <- create_round_edges(data1[(data1$round == i), ], players_in_game)
    endow <- get_endowment(data2)
    round_edges <- transform_ties_endow(round_edges,endow)
    matrices[[i]] <- round_edges
  }
  return(matrices)
}

#Create group endow x delrecip effect
transform_ties_by_endow <- function(matrix, endow, group){
  for(i in 1:nrow(matrix)){
    for(j in 1:ncol(matrix)){
      if(matrix[i,j] == 1){
        if(endow[i,"endowment"] == group){
          matrix[i,j] = 1
        }else{
          matrix[i,j] = 0  
        }
      }
    }
  }
  return(t(matrix))
}

create_group_endow_delrecip_cov <- function(data1, data2, group){
  matrices <- list()
  players_in_game <- union(unique(data1$player_id), unique(data1$other_id))
  num_players <- length(players_in_game)
  for(i in 1:20){
    round_edges <- create_round_edges(data1[(data1$round == i), ], players_in_game)
    endow <- get_endowment(data2)
    round_edges <- transform_ties_by_endow(round_edges,endow,group)
    matrices[[i]] <- round_edges
  }
  return(matrices)
}


plot_models <- function(summ, models){
  rnames <- rownames(summ)
  df <- cbind(rnames, as.data.frame.matrix(summ))[,1:3]
  model_ids <- c(which(df$rnames == "edges"), nrow(df))
  model_vec <- c()
  for(i in 1:length(model_ids)){
    if(i != length(model_ids)){
      model_vec[model_ids[i]:model_ids[i+1]] <- models[i]
    }
  }
  df$models <- model_vec
  df <- df[rnames!="edges",]
  colnames(df) <- c("term", "estimate", "std.error", "model")
  df$std.error <- as.numeric(as.character(df$std.error))
  df$estimate <- as.numeric(as.character(df$estimate))
  dwplot(df)
}

Load Data #TODO

In [7]:
interact_data <- read.csv("data/interactions.csv", header=T)
wealth_data <- read.csv("data/finwealth_from_interactions.csv", header=T)

Run Models #TODO

In [8]:
all_networks <- list()
all_models <- list()
full_summ <- NULL

for (t in unique(interact_data$treatment)) {
  group_networks <- list()
  group_models <- list()
  test <- list()
  
  for (g in unique(interact_data[(interact_data$treatment==t), ]$group_id)) {
    
    filtered_data1 <- interact_data[(interact_data$treatment==t) & (interact_data$group_id==g), ]
    filtered_data2 <- wealth_data[(wealth_data$treatment==t) & (wealth_data$group_id==g), ]
    
    btergm_network <- get_btergm_data(filtered_data1, filtered_data2)
    
    
    wealth_delrecip_cov <- create_wealth_delrecip_cov(filtered_data1,filtered_data2)
    wealth_delrecip_cov <- edgecov_add_lag(wealth_delrecip_cov, lag=1)
    
    endow_delrecip_cov2 <- create_group_endow_delrecip_cov(filtered_data1,filtered_data2,2)
    endow_delrecip_cov2 <- edgecov_add_lag(endow_delrecip_cov2, lag=1)
    endow_delrecip_cov4 <- create_group_endow_delrecip_cov(filtered_data1,filtered_data2,4)
    endow_delrecip_cov4 <- edgecov_add_lag(endow_delrecip_cov4, lag=1)
    endow_delrecip_cov6 <- create_group_endow_delrecip_cov(filtered_data1,filtered_data2,6)
    endow_delrecip_cov6 <- edgecov_add_lag(endow_delrecip_cov6, lag=1)
    
    btergm_model <- NULL
    
    if(t == "No info"){
      btergm_model <- btergm(btergm_network ~ edges + ttriple + delrecip   + memory(type = "stability")   
                             + nodeocov("endowment") + nodeocov("wealth")  , R = 100)   
    }else if(t == "Endow"){
      btergm_model <- btergm(btergm_network ~ edges + ttriple  + memory(type = "stability")     
                             + nodeocov("endowment") + nodeocov("wealth") + nodeicov("endowment")+edgecov(endow_delrecip_cov2) + edgecov(endow_delrecip_cov4)+ edgecov(endow_delrecip_cov6) , R = 100)   
    }else if(t == "Wealth"){
      btergm_model <- btergm(btergm_network ~ edges + ttriple + delrecip  + memory(type = "stability")
                             + nodeocov("endowment") + nodeocov("wealth") + nodeicov("wealth")+ edgecov(wealth_delrecip_cov), R = 100)  
    }else if(t == "Wealth + Endow"){
      btergm_model <- btergm(btergm_network ~ edges + ttriple  + memory(type = "stability")
                             + nodeocov("endowment") + nodeocov("wealth") + nodeicov("wealth") + nodeicov("endowment")+edgecov(wealth_delrecip_cov)+edgecov(endow_delrecip_cov2) + edgecov(endow_delrecip_cov4)+ edgecov(endow_delrecip_cov6), R = 100)   
    }
    
    group_networks[[paste(t, '_', g, sep="")]] <- btergm_network
    group_models[[paste(t, '_', g, sep="")]] <- btergm_model
  }
  
  all_models[[paste(t, sep="")]] <- group_models
  
  #Export metafor coefficients
  #models_summary <- summarize_models(group_models1, group_models2, group_models3)
  #df <- data.frame(matrix(unlist(models_summary), ncol=length(models_summary)))
  #write.csv(models_summary, paste('btergm_output/models_params_', t, '.csv', sep=''))

  summary <- get_metafor_data(group_models)
  write.csv(summary, paste('btergm_output/', paste('btergm_model_35',t, sep=""), '.csv', sep=''))
  full_summ <- rbind(full_summ, summary)
  #export_model(group_models, paste('btergm_model_17full',t, sep="")) 
  #all_networks[[paste(t, sep="")]] <- group_networks
}

plot_models(full_summ, unique(interact_data$treatment))





Initial dimensions of the network and covariates:


                          t=2 t=3 t=4 t=5 t=6 t=7 t=8 t=9 t=10 t=11 t=12 t=13
btergm_network (row)       20  20  20  20  20  20  20  20   20   20   20   20
btergm_network (col)       20  20  20  20  20  20  20  20   20   20   20   20
memory (row)               20  20  20  20  20  20  20  20   20   20   20   20
memory (col)               20  20  20  20  20  20  20  20   20   20   20   20
endow_delrecip_cov2 (row)  20  20  20  20  20  20  20  20   20   20   20   20
endow_delrecip_cov2 (col)  20  20  20  20  20  20  20  20   20   20   20   20
endow_delrecip_cov4 (row)  20  20  20  20  20  20  20  20   20   20   20   20
endow_delrecip_cov4 (col)  20  20  20  20  20  20  20  20   20   20   20   20
endow_delrecip_cov6 (row)  20  20  20  20  20  20  20  20   20   20   20   20
endow_delrecip_cov6 (col)  20  20  20  20  20  20  20  20   20   20   20   20
                          t=14 t=15 t=16 t=17 t=18 t=19 t=20
btergm_network (row)        20   20   20   20   20   20   20
btergm_network (col)


All networks are conformable.

Dimensions of the network and covariates after adjustment:


                          t=2 t=3 t=4 t=5 t=6 t=7 t=8 t=9 t=10 t=11 t=12 t=13
btergm_network (row)       20  20  20  20  20  20  20  20   20   20   20   20
btergm_network (col)       20  20  20  20  20  20  20  20   20   20   20   20
memory (row)               20  20  20  20  20  20  20  20   20   20   20   20
memory (col)               20  20  20  20  20  20  20  20   20   20   20   20
endow_delrecip_cov2 (row)  20  20  20  20  20  20  20  20   20   20   20   20
endow_delrecip_cov2 (col)  20  20  20  20  20  20  20  20   20   20   20   20
endow_delrecip_cov4 (row)  20  20  20  20  20  20  20  20   20   20   20   20
endow_delrecip_cov4 (col)  20  20  20  20  20  20  20  20   20   20   20   20
endow_delrecip_cov6 (row)  20  20  20  20  20  20  20  20   20   20   20   20
endow_delrecip_cov6 (col)  20  20  20  20  20  20  20  20   20   20   20   20
                          t=14 t=15 t=16 t=17 t=18 t=19 t=20
btergm_network (row)        20   20   20   20   20   20   20
btergm_network (col)


Starting pseudolikelihood estimation with 100 bootstrapping replications on a single computing core...
Done.

Initial dimensions of the network and covariates:


...


Starting pseudolikelihood estimation with 100 bootstrapping replications on a single computing core...
Done.

Initial dimensions of the network and covariates:


                          t=2 t=3 t=4 t=5 t=6 t=7 t=8 t=9 t=10 t=11 t=12 t=13
btergm_network (row)       16  16  16  16  16  16  16  16   16   16   16   16
btergm_network (col)       16  16  16  16  16  16  16  16   16   16   16   16
memory (row)               16  16  16  16  16  16  16  16   16   16   16   16
memory (col)               16  16  16  16  16  16  16  16   16   16   16   16
wealth_delrecip_cov (row)  16  16  16  16  16  16  16  16   16   16   16   16
wealth_delrecip_cov (col)  16  16  16  16  16  16  16  16   16   16   16   16
endow_delrecip_cov2 (row)  16  16  16  16  16  16  16  16   16   16   16   16
endow_delrecip_cov2 (col)  16  16  16  16  16  16  16  16   16   16   16   16
endow_delrecip_cov4 (row)  16  16  16  16  16  16  16  16   16   16   16   16
endow_delrecip_cov4 (col)  16  16  16  16  16  16  16  16   16   16   16   16
endow_delrecip_cov6 (row)  16  16  16  16  16  16  16  16   16   16   16   16
endow_delrecip_cov6 (col)  16  16  16  16  16  16  16  16   16  


All networks are conformable.

Dimensions of the network and covariates after adjustment:


                          t=2 t=3 t=4 t=5 t=6 t=7 t=8 t=9 t=10 t=11 t=12 t=13
btergm_network (row)       16  16  16  16  16  16  16  16   16   16   16   16
btergm_network (col)       16  16  16  16  16  16  16  16   16   16   16   16
memory (row)               16  16  16  16  16  16  16  16   16   16   16   16
memory (col)               16  16  16  16  16  16  16  16   16   16   16   16
wealth_delrecip_cov (row)  16  16  16  16  16  16  16  16   16   16   16   16
wealth_delrecip_cov (col)  16  16  16  16  16  16  16  16   16   16   16   16
endow_delrecip_cov2 (row)  16  16  16  16  16  16  16  16   16   16   16   16
endow_delrecip_cov2 (col)  16  16  16  16  16  16  16  16   16   16   16   16
endow_delrecip_cov4 (row)  16  16  16  16  16  16  16  16   16   16   16   16
endow_delrecip_cov4 (col)  16  16  16  16  16  16  16  16   16   16   16   16
endow_delrecip_cov6 (row)  16  16  16  16  16  16  16  16   16   16   16   16
endow_delrecip_cov6 (col)  16  16  16  16  16  16  16  16   16  


Starting pseudolikelihood estimation with 100 bootstrapping replications on a single computing core...
Done.
Warning message in btergm.se(group_models[[1]]):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) e